# Utilizing the SensorThings API
### Preliminary Steps
This script collects weather data and sends fake observations to an istSOS endpoint.

Follow the steps below to set up and execute the script:

1. Replace `IST_SOS_ENDPOINT` in the following script with your istSOS base URL (http://localhost:8018/istsos4/v1.1 or https://istsos.org/v4/v1.1).
2. Create a `sta.py` file on your machine and add the following content:

In [ ]:
import time

import requests

IST_SOS_ENDPOINT = "http://localhost:8018/istsos4/v1.1"

# Set variables from user input
istsos_username = input("Enter your istsos username: ")
if istsos_username.strip() == "":
    print("You must enter a username")
    exit()
istsos_password = input("Enter your istsos password: ")
if istsos_password.strip() == "":
    print("You must enter a password")
    exit()
datastream_id = input("Enter the datastream ID: ")
if datastream_id.strip() == "":
    print("You must enter a datastream ID")
    exit()
else:
    try:
        datastream_id = int(datastream_id)
    except ValueError:
        print("Datastream ID must be an integer")
        exit()
frequency = input(
    "Enter the frequency of the stream in seconds (default: 5): "
)
if frequency.strip() == "":
    frequency = 5
else:
    try:
        frequency = int(frequency)
    except ValueError:
        print("Frequency must be an integer")
        exit()
latitude = input("Enter the latitude of your position (default: 45.8): ")
if latitude.strip() == "":
    latitude = 45.8
else:
    try:
        latitude = float(latitude)
    except ValueError:
        print("Latitude must be a float")
        exit()
longitude = input("Enter the longitude of your position (default: 9.1): ")
if longitude.strip() == "":
    longitude = 9.1
else:
    try:
        longitude = float(longitude)
    except ValueError:
        print("Longitude must be a float")
        exit()

# basic observation schema
observation = {
    "phenomenonTime": "2015-03-03T00:00:00Z",
    "resultTime": "2015-03-03T00:00:00Z",
    "result": 3,
    "resultQuality": "100",
    "Datastream": {"@iot.id": 1},
}

# Login to istsos and get token
req = requests.post(
    f"{IST_SOS_ENDPOINT}/Login",
    headers={"Content-Type": "application/x-www-form-urlencoded"},
    data={"username": istsos_username, "password": istsos_password},
)
if req.status_code != 200:
    print("Login failed")
    exit()
else:
    print("Login successful")
token_obj = req.json()

# API to get weather data
response = requests.get(
    "https://archive-api.open-meteo.com/v1/archive",
    params={
        "latitude": latitude,
        "longitude": longitude,
        "start_date": "2024-01-21",
        "end_date": "2024-11-31",
        "temperature_unit": "celsius",
        "hourly": "temperature_2m",
    },
)
data = response.json()
temperatures = data["hourly"]
print(
    "Data received from the API: https://archive-api.open-meteo.com/v1/archive"
)

# Create a fake data
idx = 0
while True:
    observation["phenomenonTime"] = temperatures["time"][idx]
    observation["resultTime"] = temperatures["time"][idx]
    observation["result"] = temperatures["temperature_2m"][idx]
    observation["Datastream"]["@iot.id"] = datastream_id
    response = requests.post(
        f"{IST_SOS_ENDPOINT}/Observations",
        headers={
            "Content-Type": "application/json",
            "Authorization": f"Bearer {token_obj['access_token']}",
            "commit-message": "Add fake observation for tutorial",
        },
        json=observation,
    )
    if response.status_code == 201:
        print(f"Data sent: {observation}")
    else:
        print(f"Failed to send data: {observation}")
        print(response.text)
    print("Waiting for the next data...")
    print("")
    idx += 1
    time.sleep(frequency)

To start the service, navigate to the directory containing `sta.py` and execute the following command to run it in the background and begin generating data::

```sh
python3 sta.py
```

The script ask you these parameters:

- `istsos_username`: The username of the istsos user. Should have privileges to send data, likely an editor-level account.
- `istsos_password`: The password of the istsos user.
- `datastream_id`: The identifier of the datastream to which observations will be linked.
- `frequency`: The frequency of the data to be sent.
- `latitude`: The latitude of the position.
- `longitude`: The longitude of the position.


### Jupyter notebook

You can now use the Jupyter Notebook to visualize the data.

In the following script, replace `IST_SOS_ENDPOINT` variable with your istSOS base URL (http://localhost:8018/istsos4/v1.1 or https://istsos.org/v4/v1.1).

In [4]:
import requests
from matplotlib import pyplot as plt
from dateutil import parser
import requests
import time
import matplotlib.pyplot as plt
from dateutil import parser
from IPython.display import clear_output

IST_SOS_ENDPOINT = "http://localhost:8018/istsos4/v1.1"

#### Login as editor

In [8]:
# Set variables from user input
istsos_username = input("Enter your istsos username: ")
if istsos_username.strip() == "":
    print("You must enter a username")
    exit()
istsos_password = input("Enter your istsos password: ")
if istsos_password.strip() == "":
    print("You must enter a password")
    exit()
datastream_id = input("Enter the datastream ID: ")
if datastream_id.strip() == "":
    print("You must enter a datastream ID")
    exit()
else:
    try:
        datastream_id = int(datastream_id)
    except ValueError:
        print("Datastream ID must be an integer")
        exit()
frequency = input(
    "Enter the frequency of the stream in seconds (default: 5): "
)
if frequency.strip() == "":
    frequency = 5
else:
    try:
        frequency = int(frequency)
    except ValueError:
        print("Frequency must be an integer")
        exit()

# Login to istsos and get token
req = requests.post(
    f"{IST_SOS_ENDPOINT}/Login",
    headers={"Content-Type": "application/x-www-form-urlencoded"},
    data={"username": istsos_username, "password": istsos_password},
)

if req.status_code != 200:
    print("Login failed")
    raise Exception(req.text)
token_obj = req.json()

#### Retrieve data

In [ ]:
# Initial data fetch
req = requests.get(
    f"{IST_SOS_ENDPOINT}/Datastreams({datastream_id})/Observations?$orderby=phenomenonTime desc",
    headers={
        "Content-Type": "application/json",
        "Authorization": f"Bearer {token_obj['access_token']}",
    },
)

if req.status_code != 200:
    print("Failed to get datastream")
    raise Exception(req.text)
else:
    data = req.json()["value"]
    dt = [parser.parse(i["phenomenonTime"]) for i in data]
    values = [i["result"] for i in data]

# Reverse the order for chronological plotting
dt.reverse()
values.reverse()


while True:

    # Fetch the latest observation
    req = requests.get(
        f"{IST_SOS_ENDPOINT}/Datastreams({datastream_id})/Observations?$orderby=phenomenonTime desc",
        headers={
            "Content-Type": "application/json",
            "Authorization": f"Bearer {token_obj['access_token']}",
        },
    )
    if req.status_code == 200:

        clear_output(wait=True)
        fig, ax = plt.subplots(figsize=(20, 10))
        ax.set_title("Real-Time Sensor Data")
        ax.set_xlabel("Time")
        ax.set_ylabel("Values")
        ax.grid(True)
        new_data = req.json()["value"]
        for obs in new_data:
            obs_time = parser.parse(obs["phenomenonTime"])
            if obs_time not in dt:
                dt.append(obs_time)
                values.append(obs["result"])
        ax.plot(dt, values, label="Sensor Data")
        plt.pause(0.1)

    else:
        print("Failed to fetch the latest observation")
        print(req.text)

    time.sleep(frequency)